In [3]:
!pip install pyspark

from google.colab import drive
drive.mount('/content/drive')

import pyspark.sql.functions as f
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, IntegerType, StringType, FloatType, StructField,BooleanType

schema = StructType([
    StructField("CrimeRate", FloatType(), True), 
    StructField("Youth", IntegerType(), True), 
    StructField("Southern", IntegerType(), True), 
    StructField("Education", FloatType(), True), 
    StructField("ExpenditureYear0", FloatType(), True), 
    StructField("LabourForce", IntegerType(), True), 
    StructField("Males", IntegerType(), True), 
    StructField("MoreMales", IntegerType(), True), 
    StructField("StateSize", IntegerType(), True), 
    StructField("YouthUnemployment", IntegerType(), True), 
    StructField("MatureUnemployment", IntegerType(), True),
    StructField("HighYouthUnemploy", IntegerType(), True),
    StructField("Wage", IntegerType(), True),
    StructField("BelowWage", IntegerType(), True)
    ])

spark = SparkSession.builder.appName("Сrime").getOrCreate()
df = spark.read.format("csv").option("header", True).schema(schema).load("Crime.csv")
df.printSchema()
df.show()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
root
 |-- CrimeRate: float (nullable = true)
 |-- Youth: integer (nullable = true)
 |-- Southern: integer (nullable = true)
 |-- Education: float (nullable = true)
 |-- ExpenditureYear0: float (nullable = true)
 |-- LabourForce: integer (nullable = true)
 |-- Males: integer (nullable = true)
 |-- MoreMales: integer (nullable = true)
 |-- StateSize: integer (nullable = true)
 |-- YouthUnemployment: integer (nullable = true)
 |-- MatureUnemployment: integer (nullable = true)
 |-- HighYouthUnemploy: integer (nullable = true)
 |-- Wage: integer (nullable = true)
 |-- BelowWage: integer (nullable = true)

+---------+-----+--------+---------+----------------+-----------+-----+---------+---------+-----------------+------------------+-----------------+----+---------+
|C

In [10]:
# Знайти середню частоту злочинів в південних штатах

df.select(df.CrimeRate,df.Southern).where(df.Southern > 0).agg(f.avg('CrimeRate').alias('avg')).show()

+------------------+
|               avg|
+------------------+
|100.68124985694885|
+------------------+



In [11]:
# Порахувати загальні витрати на поліцію

df.groupBy().sum('ExpenditureYear0').collect()


[Row(sum(ExpenditureYear0)=3995.0)]

In [13]:
# Вивести частоту злочинів для північних штатів, в яких кількість бідних сімей більше 200 на 1000 нас

df.select(df.CrimeRate,df.Southern,df.BelowWage).where(df.Southern > 0).where(df.BelowWage > 200).show()

+---------+--------+---------+
|CrimeRate|Southern|BelowWage|
+---------+--------+---------+
|     56.6|       1|      276|
|     60.3|       1|      249|
|     79.8|       1|      250|
|     85.6|       1|      237|
|     96.8|       1|      261|
|     97.4|       1|      264|
|     98.7|       1|      224|
|     99.9|       1|      251|
|    103.0|       1|      254|
|    106.6|       1|      239|
|    112.1|       1|      247|
|    121.6|       1|      233|
|    123.4|       1|      228|
|    145.4|       1|      206|
+---------+--------+---------+



In [16]:
# Відсортувати дані за кількістю безробітних чоловіків віком 18-24 роки на 1000 нас (зростаючий)

df.sort(df.YouthUnemployment.asc()).show()

+---------+-----+--------+---------+----------------+-----------+-----+---------+---------+-----------------+------------------+-----------------+----+---------+
|CrimeRate|Youth|Southern|Education|ExpenditureYear0|LabourForce|Males|MoreMales|StateSize|YouthUnemployment|MatureUnemployment|HighYouthUnemploy|Wage|BelowWage|
+---------+-----+--------+---------+----------------+-----------+-----+---------+---------+-----------------+------------------+-----------------+----+---------+
|     70.5|  130|       0|     14.1|            63.0|        641|  984|        0|       14|               70|                21|                1| 486|      196|
|     85.6|  166|       1|     11.4|            58.0|        521|  973|        0|       46|               72|                26|                0| 396|      237|
|     98.7|  162|       1|     12.1|            75.0|        522|  996|        0|       40|               73|                27|                0| 496|      224|
|    103.0|  177|       1|  

In [15]:
#Відсортувати дані за середніми доходами (спадний)

df.sort(df.Wage.desc()).show()


+---------+-----+--------+---------+----------------+-----------+-----+---------+---------+-----------------+------------------+-----------------+----+---------+
|CrimeRate|Youth|Southern|Education|ExpenditureYear0|LabourForce|Males|MoreMales|StateSize|YouthUnemployment|MatureUnemployment|HighYouthUnemploy|Wage|BelowWage|
+---------+-----+--------+---------+----------------+-----------+-----+---------+---------+-----------------+------------------+-----------------+----+---------+
|     84.9|  121|       0|     13.2|           118.0|        547|  964|        0|       25|               84|                29|                0| 689|      126|
|    161.8|  131|       0|     13.2|           160.0|        631| 1071|        1|        3|              102|                41|                0| 674|      152|
|    157.7|  136|       0|     15.1|           149.0|        577|  994|        0|      157|              102|                39|                0| 673|      167|
|    154.3|  124|       0|  